<a href="https://colab.research.google.com/github/kr7/IntelligensModszerekTantargy/blob/main/CleanCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Clean Code és Unit tesztek**

**Clean Code** 

- *A kód olvasásával és írásával töltött idő aránya jelentősen több, mint 10:1-hez, az olvasás javára* (Robert C. Martin: Clean Code)

- Azt szeretnénk, hogy általunk írt kód jól olvasható, könnyen értelmezhető legyen. Miért? 

  - a kód helyességének ellenőrzése, biztosítása

  - hosszú távú karbantarthatóság

- Kerüljük a nehezen értelmezhető, nem-intuitív, akár félrevezető kódrészeket. Az alábbiak közül melyik olvasható jobban?
    

          legkondi_allapota = "off" or legkondi_allapota
          
          legkondi_allapota = legkondi_allapota or "off"
         
          if legkondi_allapota is None:
            legkondi_allapota = "off"

              
          lista.append(elem)

          lista = lista + [elem] 

          


- Elnevezések: használjunk beszédes elnevezéseket, olyan elővigyázatossággal járjunk el a változók, függvények, osztályok, példányok, stb. elnevezésekor, mintha a saját gyerekünknek adnánk nevet - Elon Musk ellenpélda! :-) 

- Amit csak lehet, nevezzünk el, ne használjunk "mágikus konstansokat", hanem nevezzük el a számokat


          k = 2 * 31.415
          
          kor_sugara = 10
          PI = 3.1415
          kor_kerulete = 2 * PI * kor_sugara
          

          FEHER = 0
          FEKETE = 1
          ...
          mezo_allapota = FEHER


- Rövid, áttekinthető függvényeket, osztályokat írjunk, egy függvény/osztály egy dolgot csináljon! Inkább több rövid (és jól elnevezett függvényt/osztályt) írjunk, mint kevés hosszú, nehezen áttekinthető függvényt/osztályt!

- A logikailag összetartozó "dolgok" egymáshoz közel legyenek a kódban, tagoljuk a kódot (üres sorok, szóközök)!

- Ne használjunk túl hosszú sorokat, sokszorosan egymásba ágyazott zárójeleket!

- Használjunk típusannotációkat!

- Kommentek: főleg "miért"-re fókuszáljon, a "mit"/"hogyan" legyen érthető a kódból ("self-documenting code").

- Hibák kezelése: 

  - legyünk "őszinték", ne hagyjuk figyelmen kívül al hibát ("a legkissebb hibád, megbosszúlja önmagát" - Máté Péter: Most élsz),
  
  - használjunk (saját) Exception-öket és beszédes hibaüzeneteket!

- Használjunk unit teszteket (erről kicsit részletesebben lesz szó a következő szakaszban)!

- ... (a felsorolás még sokáig folytatható, ld. Robert C. Martin: Clean Code c. könyvét) ...





**Példa**

A legelső órán láttunk egy ún. *tökéletes számokat* kereső kódot, ami egyébként hibás, de a hibát (ebben az alakban) nehéz megtalálni. Ezért most refaktoráljuk a kódot. (Emlékeztetőül: tökéletes számnak nevezünk egy számot akkor, ha az osztóinak összege, önmagát nem számítva, megegyezik a számmal. Például: 6 tökéletes, hiszen 1+2+3=6.)

In [ ]:
import math

def tokeletes_e(szam):
  osztok_osszege = 1
  for x in range(2,math.ceil(math.sqrt(szam))):
    if szam % x == 0:
      osztok_osszege += x
      osztok_osszege += (szam/x)
  if osztok_osszege == szam:
    return True
  else:
    return False

for i in range(6,10000+1):
  if tokeletes_e(i):
    print(i)

6
28
496
8128


In [ ]:
def osztoja(szam: int, lehetseges_oszto: int) -> bool:
  return (szam % lehetseges_oszto == 0)


def legnagyobb_vizsgalando_oszto(szam: int) -> int:
  return math.ceil(math.sqrt(szam)) - 1


def szam_osztoinak_osszege(szam: int) -> int:
  osztok_osszege = 1
  for lehetseges_oszto in range(2, legnagyobb_vizsgalando_oszto(szam) + 1 ):
    if osztoja(szam, lehetseges_oszto):
      osztok_osszege += lehetseges_oszto
      osztok_osszege += (szam/lehetseges_oszto)
  return osztok_osszege


def tokeletes_e(szam: int) -> bool:
  return szam_osztoinak_osszege(szam) == szam


def tokeletes_szamok_keresese(meddig: int):
  for i in range(meddig+1):
    if tokeletes_e(i):
      print(i)


In [ ]:
tokeletes_szamok_keresese(10000)

1
6
28
496
8128


A kód még mindig hibás (a refaktorálás során ekvivalens átalakításokat végeztünk), de immár van esélyünk a hibát észrevenni unit tesztek segítségével.

**Unit tesztek**

- Alapgondolat: megnézzük, hogy a függvényeink adott bementre az "elvárt" kimenetet adják-e.

- Automatizált tesztek: tetszőlegesen sokszor futtatható, dokumentált, reprodukálható, jól elkészített unit tesztekkel a bugok és nem kívánt változások (akár interpreter viselkedésének változása is!) jelentős  része felismerhető.

- Unit tesztek tipikus szerkezete: függvényhívás előkészítése (pl. szükséges objektumok példányosítása), függvény hívása, visszaadott érték ellenőrzése.

- A tesztek legyenek lehetőleg egyszerűek, lehetőleg minden releváns esetet teszteljünk, a lehetséges végrehajtási utakat járjuk be.

- A unit tesztek **nem garantálják** a program helyességét! (Még 100%-os lefedettség, azaz az összes lehetséges végrehajtási út bejárása esetén sem!)

- A függvények gyakran egy összetett rendszer részei, a rendszeren belül "léteznek", azaz ahhoz, hogy el tudják végezni a kívánt műveleteket egy komplexebb környezetre (pl. adatbázisszerver, web szerver, stb.) lehet szükség. Ilyen esetkben hasznos az ún. "mockolás" (mocking), amellyel tudjuk ellenőrizni, például hogy egy függvény megfelelő parancsokat küld-e egy adatbázisszervernek anélkül, hogy létrehoznánk az adatbázisszervert. 

- Unit teszetelésre több keretrendszer is létezik (az érdeklődőknek ajánljuk ezek tanulmányozását), a következő példák csak a tesztek jellegét illusztrálják egyszerű esetekben, nem mutatják be egy konkrét keretrendszer használatát.

In [ ]:
def test_osztoja_10_5():
  szam = 10
  lehetseges_oszto = 5
  elvart_valasz = True

  kapott_valasz = osztoja(szam, lehetseges_oszto)

  assert kapott_valasz == elvart_valasz

def test_osztoja_10_3():
  szam = 10
  lehetseges_oszto = 3
  elvart_valasz = False

  kapott_valasz = osztoja(szam, lehetseges_oszto)
  assert kapott_valasz == elvart_valasz

In [ ]:
test_osztoja_10_5()

In [ ]:
test_osztoja_10_3()

In [ ]:
def test_legnagyobb_vizsgalando_oszto_10():
  szam = 10
  elvart_valasz = 3

  kapott_valasz = legnagyobb_vizsgalando_oszto(szam)
  
  assert kapott_valasz == elvart_valasz

In [ ]:
test_legnagyobb_vizsgalando_oszto_10()

In [ ]:
def test_legnagyobb_vizsgalando_oszto_25():
  szam = 25
  elvart_valasz = 5

  kapott_valasz = legnagyobb_vizsgalando_oszto(szam)
  
  assert kapott_valasz == elvart_valasz

In [ ]:
test_legnagyobb_vizsgalando_oszto_25()

AssertionError: ignored

In [ ]:
legnagyobb_vizsgalando_oszto(25)

4

In [ ]:
def test_szam_osztoinak_osszege_6():
  szam = 6
  elvart_valasz = 6

  kapott_valasz = szam_osztoinak_osszege(szam)

  assert kapott_valasz == elvart_valasz

In [ ]:
def test_szam_osztoinak_osszege_25():
  szam = 25
  elvart_valasz = 6

  kapott_valasz = szam_osztoinak_osszege(szam)

  assert kapott_valasz == elvart_valasz

In [ ]:
test_szam_osztoinak_osszege_6()

In [ ]:
test_szam_osztoinak_osszege_25()

AssertionError: ignored

In [ ]:
szam_osztoinak_osszege(25)

1

Javítjuk a legnagyobb viszgálandó osztót meghatározó kódot.

In [ ]:
def legnagyobb_vizsgalando_oszto(szam: int) -> int:
  return math.floor(math.sqrt(szam)) 

In [ ]:
test_legnagyobb_vizsgalando_oszto_10()
test_legnagyobb_vizsgalando_oszto_25()

In [ ]:
test_szam_osztoinak_osszege_6()

In [ ]:
test_szam_osztoinak_osszege_25()

AssertionError: ignored

In [ ]:
szam_osztoinak_osszege(25)

11.0

A hiba az, hogy amikor egy négyzetszám osztóit vizsgáljuk, a szám gyökét kétszer is hozzáadjuk az osztók összegéhez. Javítjuk az osztók összegét meghatározó kódot is.

In [ ]:
def szam_osztoinak_osszege(szam: int) -> int:
  osztok_osszege = 1
  for lehetseges_oszto in range(2, legnagyobb_vizsgalando_oszto(szam) + 1 ):
    if osztoja(szam, lehetseges_oszto):
      osztok_osszege += lehetseges_oszto
      masik_oszto = (szam/lehetseges_oszto)
      if masik_oszto != lehetseges_oszto:
        osztok_osszege += masik_oszto
  return osztok_osszege

In [ ]:
test_szam_osztoinak_osszege_25()

A biztonság kedvéért lefuttatjuk az összes unit tesztünket, hogy kiderüljön, nem rontottunk-e el valami mást a javítások közben.

In [ ]:
test_osztoja_10_3()

In [ ]:
test_osztoja_10_5()

In [ ]:
test_legnagyobb_vizsgalando_oszto_10()

In [ ]:
test_legnagyobb_vizsgalando_oszto_25()

In [ ]:
test_szam_osztoinak_osszege_6()

In [ ]:
test_szam_osztoinak_osszege_25()

Ami a megtalált tökéletes számokat illeti, a hiba javításának nincs hatása az eredményre:

In [ ]:
tokeletes_szamok_keresese(10000)

1
6
28
496
8128
